# Bigram-gram based LLM

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset
import torch
import re


## Dataset Preprocessing

In [2]:
ds = load_dataset("MuskumPillerum/General-Knowledge")['train']

In [23]:
l = []
for i,j in zip(ds['Question'], ds['Answer']):
    if i and j:
        i = re.sub(r'[^\w\s]', '', i).replace('\\n', "") + '?'
        # i = ''.join(i.replace(".", "").replace("'", "").replace('"', "").replace('\\n', "").split('?')) + ' ?'
        j = j.replace('\\n', "")
        l.append(f'<Start> Question: {i.lower()} Answer: {j.lower()} <Stop>')

    # break

In [4]:
vocab = []
for sentence in l:
    vocab.extend(sentence.split())

len(vocab)


2423271

In [21]:
import re

text = "Hello,World! How are you?"
text_without_punctuation = re.sub(r'[^\w\s]', '', text)

print(text_without_punctuation)

HelloWorld How are you


In [24]:
vocab

{'balls.',
 'homo.2.6',
 'soft.-',
 'home6.',
 '0s',
 '"aerial',
 'labelled,',
 'cathedral,',
 'profilers',
 'derivatives',
 'searchable',
 'kingdom.',
 'vera',
 'stranded',
 'commented',
 'it.step',
 'apes',
 'glycolysis,',
 'while(true):',
 'liu',
 'water-borne',
 'frayswhen',
 'sierra',
 'good5',
 'lee.',
 'livechorus',
 'montana,',
 'bot’s',
 'flamboyant',
 'yoshi,',
 'drain.',
 '(0.93%),',
 '8πt,',
 'maxvalue',
 '(y/z).',
 'oligopoly',
 '"throw"',
 'cyanobacteria,',
 'greatly,',
 'tallied',
 'preserves',
 'firewood.',
 '"inevitable".',
 'success.throughout',
 'meal-planning',
 '"diamond',
 'skunk.',
 'chloroplasts',
 'gasped',
 'expanding.',
 'searchlist',
 '(ivt)',
 'pseudorandom',
 'and7.',
 'hiking',
 'poland1940:-',
 'talkingshow',
 'night"',
 'codebase.3.',
 'd;',
 'distant.',
 'battered.',
 '9,400,000',
 'loops.7.',
 '4:clouds',
 'school-age',
 'paper-',
 'trains.',
 'dom',
 '4[the',
 'misguidedbut',
 'veil',
 'ovation',
 'nepal-',
 'modify.',
 'processes-',
 'underwater?"',

In [5]:
vocab = set(vocab)

vocab_dict = {} 
ct = 0
for i in vocab:
    vocab_dict[i] = ct
    ct += 1
print(ct)

106906


In [6]:
rev_vocab_dict = {v:k for k,v in vocab_dict.items()}

## Model Creation

In [7]:
n_gram_matrix = np.ones((len(vocab), len(vocab)), dtype=np.int8) # 15 gig!
n_gram_matrix.shape

(106906, 106906)

In [8]:
n_gram_matrix

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int8)

In [10]:
for sentence in l:
    prev_word_index = vocab_dict['<Start>']
    # print(sentence)
    for word in sentence.split()[1:]:
        index = vocab_dict[word]
        n_gram_matrix[prev_word_index][index] += 1 # So that it is bigger than 1
        prev_word_index = index


In [11]:
l = np.ones(10, dtype=np.int8)

In [12]:
l

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int8)

In [13]:
for r in n_gram_matrix:
    if max(r)<=0 or min(r) <=0:
        # print('cow')
        print(min(r))

-92
-68
-26
-71
-123
-99


KeyboardInterrupt: 

In [ ]:
max(r), min(r)

(11, 1)

# Inference

In [ ]:
out = vocab_dict['<Start>']
len = 0
while out != vocab_dict['<Stop>']:
    row = torch.tensor(n_gram_matrix[out], dtype=torch.float32)
    # row = row/ row.sum()
    
    print(row)
    next_index = torch.multinomial(torch.tensor(row, dtype=torch.float16), num_samples=1, replacement=True)
    print(next_index)
    print(rev_vocab_dict[next_index.item()], end=' ')
    out = next_index
    len += 1
    if len > 30: break
    break

tensor([1., 1., 1.,  ..., 1., 1., 1.])


/tmp/ipykernel_27458/2749186331.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  next_index = torch.multinomial(torch.tensor(row, dtype=torch.float16), num_samples=1, replacement=True)


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

In [ ]:
max(row), min(row)

(tensor(1.), tensor(-89.))

In [ ]:
min(row)

-0.0008332085080886758

In [ ]:
for i in row: print(i)

9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.346667912889055e-06
9.34666791